In [1]:
import json
import requests
import time
from datetime import datetime as dt
import hmac
import hashlib
import base64
from biscoint_api_python import Biscoint
from dotenv import load_dotenv
import threading
from playsound import playsound
import utils
load_dotenv()

True

In [3]:
# Constants
BASE_URL="https://api.biscoint.io/v1"
API_KEY = os.getenv('API_KEY') 
API_SECRET= os.getenv('API_SECRET') 
AMOUNT = '0.00001'
MIN_PERCENT_REQUIRED = 0.2

In [4]:
bsc = Biscoint(API_KEY, API_SECRET)
endpoints_meta = bsc.get_meta()
rate_limit_offer = endpoints_meta['endpoints']['offer']['post']['rateLimit']
sleep_time_offers = ((rate_limit_offer["windowMs"] / rate_limit_offer["maxRequests"]) / 1000)*2
initial_balance = bsc.get_balance()
last_balance = initial_balance

In [6]:
# Calulate the percent
percent = lambda   value1, value2  :  round((float(value2) / float(value1) - 1) * 100,3)
# minimal amount of br money to buy bitcoin
can_buy = lambda brl,min_brl=10 : float(brl) >= float(min_brl)

In [8]:
#requests.get(f"{BASE_URL}/fees").json()
bsc.get_fees()

{'withdrawal': {'BTC': {'rate': '0.0',
   'fixed': {'slow': '0.00037', 'normal': '0.00046', 'fast': '0.00054'}},
  'BRL': {'rate': '0.0',
   'fixed': {'ted': '14.90', 'sameBankTransfer': '14.90'}}}}

In [9]:
last_balance['BRL']

'0.00'

In [62]:
print(buy['efPrice'])
print(sell['efPrice'])

322000.00
320000.00


In [63]:
sell

{'offerId': 'SrkvzpPGohnsL7HWp',
 'base': 'BTC',
 'quote': 'BRL',
 'op': 'sell',
 'isQuote': False,
 'baseAmount': '0.00001000',
 'quoteAmount': '3.20',
 'efPrice': '320000.00',
 'createdAt': '2021-04-18T03:25:09.768Z',
 'expiresAt': '2021-04-18T03:25:24.768Z',
 'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}

In [58]:
while True:
    start_time = dt.now()
    buy = bsc.get_offer(op='buy',amount=AMOUNT,isQuote=False)
    sell = bsc.get_offer(op='sell',amount=AMOUNT,isQuote=False)

    calculated_percent = percent(buy['efPrice'],sell['efPrice'])
    print(f"Percent is {calculated_percent}")

    if(calculated_percent >= MIN_PERCENT_REQUIRED):
        print(f"Arbitrage oportunity: buy:{buy['efPrice']}   sell:{sell['efPrice']}")
        playsound('beep.wav')

        if not can_buy(last_balance['BRL']):
            executed_sell = bsc.confirm_offer(sell['offerId'])
            executed_buy = bsc.confirm_offer(buy['offerId'])
            last_balance = bsc.get_balance()
            break




    end_time = dt.now()
    seconds_elapsed = (end_time - start_time).total_seconds()

    print(f"Took {seconds_elapsed} seconds")
    
    time.sleep(sleep_time_offers)

Percent is -0.935
Took 3.163435 seconds
Percent is -0.621
Took 3.899339 seconds
Percent is -0.623
Took 2.410307 seconds
Percent is -0.312
Took 1.934181 seconds
Percent is -0.623
Took 2.674498 seconds
Percent is -0.623
Took 3.474864 seconds
Percent is -0.621
Took 2.347088 seconds


KeyboardInterrupt: 

In [11]:
sell =bsc.get_offer(op='sell',amount=AMOUNT,isQuote=False)
print (sell)
bsc.confirm_offer(sell['offerId'])

{'offerId': 'RoThtZZm2gsdrEcyt', 'base': 'BTC', 'quote': 'BRL', 'op': 'sell', 'isQuote': False, 'baseAmount': '0.00001000', 'quoteAmount': '3.15', 'efPrice': '315000.00', 'createdAt': '2021-04-18T19:11:57.258Z', 'expiresAt': '2021-04-18T19:12:12.258Z', 'apiKeyId': 'vo3Jt2cEmvPxQZ7NS'}


HTTPError: 403 Client Error: Forbidden for url: https://api.biscoint.io/v1/offer/confirm

In [43]:
import threading
import random


def threadfunc(name):
    random_time = random.randint(1,5)
    print(name,'Started',str(random_time),'time')
    time.sleep(random_time) # sleep to make the "work" take longer
    print(name,'Finished')
    return "Result " + name

# Collect the threads
threads = []
for i in range(0,2):
    thread = threading.Thread(target=threadfunc,args=(str(i)))
    threads.append(thread)

# Start them all
for thread in threads:
    thread.start()

# Wait for all to complete
for thread in threads:
    res = thread.join()
    print(res)
    
print("All Finished")

0 Started 4 time
1 Started 2 time
1 Finished
0 Finished
None
None
All Finished


In [285]:
r = requests.post(f'{BASE_URL}/balance',headers=headers('balance'))#.json()
r.json()

{'code': 4003,
 'error': 'Forbidden',
 'details': 'Invalid authorization headers'}

In [65]:
from playsound import playsound
playsound('beep.wav')

In [ ]:
#thread_buy = threading.Thread(target=offerFunc,args=('buy',AMOUNT,False,buy))
#thread_sell = threading.Thread(target=offerFunc,args=('sell',AMOUNT,False,sell))
#thread_buy.start()
#thread_sell.start()
#thread_buy.join()
#thread_sell.join()
